In [6]:
import pandas as pd
import re

# Função para converter string de tempo para horas decimais
def time_to_decimal(time_str):
    if pd.isnull(time_str):  # Verifica se a string é NaN ou None
        return 0
    factors = {'d': 24*60, 'h': 60, 'm': 1, 's': 1/60}
    matches = re.findall(r'(\d+)([dhms])', time_str)
    return sum(int(value) * factors[unit] for value, unit in matches)

# Função para converter string de tempo para minutos
def time_to_minutes(time_str):
    return time_to_decimal(time_str)  # Como a função já retorna minutos, podemos reutilizá-la

# Carrega o arquivo Excel, ajustando a linha do cabeçalho caso necessário
df = pd.read_excel('/content/entrada_mod.xlsx', sheet_name='Sheet1', header=1)

# Verifica se a coluna "Duration" está presente
if 'Duration' not in df.columns:
    raise KeyError('A coluna "Duration" não foi encontrada no DataFrame.')

# Aplica a conversão à coluna "Duration"
df['Duration Minutes'] = df['Duration'].apply(time_to_minutes)  # Converte para minutos
df['Duration Decimal'] = df['Duration Minutes'] / 60  # Converte para horas decimais

# Carrega o arquivo Excel com as informações de gerência
df_perfis = pd.read_excel('Perfis_iEnergy_dez23_unhide2.xls', sheet_name='iEnergy')

# Cria uma coluna "GERENCIA" no DataFrame principal
df['GERÊNCIA'] = None

# Percorre os registros do DataFrame principal
for i in range(len(df)):
    # Obtém o nome do usuário
    user = df.loc[i, 'User']

    # Encontra o registro correspondente no DataFrame de perfis
    df_perfis_row = df_perfis.loc[df_perfis['NOME'] == user]

    # Se o registro for encontrado, atribui a gerência
    if len(df_perfis_row) > 0:
        df.loc[i, 'GERÊNCIA'] = df_perfis_row['GERÊNCIA'].values[0]

# Create summary DataFrames
summary_by_user = df.groupby('User')['Duration Minutes'].sum().reset_index()
summary_by_state = df.groupby('State')['Duration Minutes'].sum().reset_index()
summary_by_user_and_state = df.groupby(['User', 'State'])['Duration Minutes'].sum().reset_index()

# Write to Excel
with pd.ExcelWriter('saidaVF.xlsx') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    summary_by_user.to_excel(writer, index=False, sheet_name='Summary By User')
    summary_by_state.to_excel(writer, index=False, sheet_name='Summary By State')
    summary_by_user_and_state.to_excel(writer, index=False, sheet_name='Summary By User And State')
    summary_by_gerencia_and_state.to_excel(writer, index=False, sheet_name='Summary By Gerencia And State')

# O arquivo 'saida.xlsx' terá várias guias:
# 'Sheet1' com os dados originais e colunas adicionais,
# 'Summary By User' com o somatório do tempo por usuário,
# 'Summary By State' com o somatório do tempo por estado,
# 'Summary By User And State' com as colunas User, State Finished, State Killed e Duration Minutes.
# 'Summary By Gerencia And State' com as colunas Gerencia, State e Duration Minutes.